In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import idx2numpy

## Prepare Data

#### Read the data from idx files

In [2]:
filename = {'train_images' : './Dataset/train-images-idx3-ubyte', 'test_images' : './Dataset/t10k-images-idx3-ubyte','train_labels' : './Dataset/train-labels-idx1-ubyte', 'test_labels' : './Dataset/t10k-labels-idx1-ubyte'}
train_imagesfile = open(filename['train_images'],'rb')
train_labelsfile = open(filename['train_labels'],'rb')
test_imagesfile = open(filename['test_images'],'rb')
test_labelsfile = open(filename['test_labels'],'rb')

#### Convert the files from idx to numpy

In [3]:
dataset_images = idx2numpy.convert_from_file(train_imagesfile)
dataset_labels = idx2numpy.convert_from_file(train_labelsfile)
dataset_images_test = idx2numpy.convert_from_file(test_imagesfile)
dataset_labels_test = idx2numpy.convert_from_file(test_labelsfile)

#### Create tensors for input into the network

In [4]:
dataset_images = dataset_images.reshape((60000, 28, 28, 1))
dataset_images_test = dataset_images_test.reshape((10000, 28, 28, 1))

# Normalize pixel values to be between 0 and 1
dataset_images, dataset_images_test = dataset_images / 255.0, dataset_images_test / 255.0

## Create the CNN

In [5]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

W0821 14:14:46.242675 140483041191744 deprecation.py:506] From /home/chc/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


#### Add Output Layers

In [7]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

## Training

In [9]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(dataset_images, dataset_labels, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 10s 168us/sample - loss: 0.1416 - acc: 0.9566
Epoch 2/5
60000/60000 [==============================] - 10s 167us/sample - loss: 0.0465 - acc: 0.9851
Epoch 3/5
60000/60000 [==============================] - 10s 167us/sample - loss: 0.0328 - acc: 0.9894
Epoch 4/5
60000/60000 [==============================] - 10s 169us/sample - loss: 0.0261 - acc: 0.9913
Epoch 5/5
60000/60000 [==============================] - 10s 168us/sample - loss: 0.0197 - acc: 0.9937


## Testing

In [10]:
test_loss, test_acc = model.evaluate(dataset_images_test, dataset_labels_test)

10000/10000 [==============================] - 0s 46us/sample - loss: 0.0279 - acc: 0.9923


## Accuracy

In [11]:
print(test_acc)

0.9923
